# Purpose


Test a few more classification metrics to include to select the best models based on these.

# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from datetime import datetime
import gc
import os
import logging
from pathlib import Path
from pprint import pprint

import mlflow

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import seaborn as sns


import subclu

from subclu.utils import set_working_directory
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric
)
from subclu.utils.mlflow_logger import MlflowLogger, log_clf_report_and_conf_matrix

from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl
)


print_lib_versions([mlflow, np, pd, plotly, sns, subclu])

python		v 3.7.10
===
mlflow		v: 1.16.0
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
seaborn		v: 0.11.1
subclu		v: 0.4.1


In [12]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Test additional classification metrics

In [13]:
import random

from sklearn.metrics import (
    classification_report,
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix
)

from subclu.utils import ml_metrics

In [14]:
n_array_test = 99
y_true = np.random.choice(['sports', 'politics', 'gaming', 'anime', 'music'], size=n_array_test, 
                          p=[.2, .3, .4, .05, .05])
y_pred = [random.choice(['sports', 'politics', 'gaming', ] + [v] * 2) for v in y_true]

In [24]:
y_true_binary = np.random.choice(
    ['sports', 'gaming'], size=n_array_test, 
    p=[.8, .2]
)
y_pred_binary = [random.choice(['sports'] * 2 + [v] * 1) for v in y_true_binary]

In [25]:
y_true_manual = [
    'politics', 'politics', 'anime', 'politics', 'gaming', 'sports',
    'gaming', 'politics', 'politics', 'gaming', 'gaming', 'gaming',
    'politics', 'politics', 'sports', 'gaming', 'sports', 'politics',
    'sports', 'sports', 'politics', 'gaming', 'gaming', 'gaming',
    'politics', 'gaming', 'music', 'sports', 'sports', 'gaming',
    'politics', 'gaming', 'politics', 'politics', 'politics',
    'politics', 'music', 'politics', 'politics', 'sports', 'sports',
    'gaming', 'gaming', 'politics', 'sports', 'gaming', 'politics',
    'sports', 'politics', 'music', 'politics', 'politics', 'gaming',
    'politics', 'gaming', 'gaming', 'gaming', 'gaming', 'gaming',
    'politics', 'sports', 'gaming', 'politics', 'anime', 'gaming',
    'politics', 'sports', 'politics', 'sports', 'gaming', 'gaming',
    'gaming', 'music', 'gaming', 'gaming', 'gaming', 'politics',
    'politics', 'sports', 'gaming', 'gaming', 'politics', 'gaming',
    'gaming', 'politics', 'gaming', 'music', 'sports', 'politics',
    'sports', 'gaming', 'sports', 'sports', 'gaming', 'politics',
    'gaming', 'gaming', 'music', 'politics'
]

y_pred_manual = [
    'politics', 'politics', 'politics', 'politics', 'gaming', 'sports',
    'gaming', 'politics', 'politics', 'gaming', 'politics', 'politics',
    'politics', 'politics', 'gaming', 'gaming', 'politics', 'politics',
    'sports', 'sports', 'politics', 'politics', 'gaming', 'gaming',
    'politics', 'gaming', 'sports', 'sports', 'sports', 'politics',
    'gaming', 'gaming', 'gaming', 'sports', 'sports', 'politics',
    'sports', 'politics', 'politics', 'politics', 'sports', 'gaming',
    'gaming', 'gaming', 'sports', 'gaming', 'politics', 'sports',
    'politics', 'gaming', 'gaming', 'sports', 'politics', 'politics',
    'politics', 'politics', 'gaming', 'gaming', 'gaming', 'politics',
    'sports', 'gaming', 'politics', 'sports', 'sports', 'sports',
    'sports', 'politics', 'sports', 'gaming', 'gaming', 'gaming',
    'politics', 'politics', 'gaming', 'sports', 'sports', 'gaming',
    'sports', 'gaming', 'gaming', 'sports', 'gaming', 'sports',
    'sports', 'gaming', 'gaming', 'sports', 'sports', 'sports',
    'gaming', 'sports', 'gaming', 'politics', 'gaming', 'gaming',
    'gaming', 'music', 'sports'
]

## Log classification report AND confusion matrix in one call

In [31]:
print(classification_report(y_true_manual, y_pred_manual, digits=3, zero_division=0))

              precision    recall  f1-score   support

       anime      0.000     0.000     0.000         2
      gaming      0.722     0.684     0.703        38
       music      1.000     0.167     0.286         6
    politics      0.594     0.559     0.576        34
      sports      0.500     0.789     0.612        19

    accuracy                          0.616        99
   macro avg      0.563     0.440     0.435        99
weighted avg      0.638     0.616     0.602        99



In [32]:
# with sort_labels_by_support=False, we should get the same class order as classification report
ml_metrics.log_classification_report_and_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    class_labels=None,
    sort_labels_by_support=False,
    save_path='mlruns/logs/test_classification_report3',
    log_metrics_to_console=False,
    log_df_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_support_avg=False,
    log_support_per_class=False,
)

01:48:46 | INFO | "df metrics:
      class  precision    recall  f1_score  support data_fold
0     anime   0.000000  0.000000  0.000000        2      k100
1    gaming   0.722222  0.684211  0.702703       38      k100
2     music   1.000000  0.166667  0.285714        6      k100
3  politics   0.593750  0.558824  0.575758       34      k100
4    sports   0.500000  0.789474  0.612245       19      k100"
01:48:46 | INFO | "df metrics:
          class  precision    recall  f1_score  support data_fold
0     macro_avg   0.563194  0.439835  0.435284       99      k100
1  weighted_avg   0.637696  0.616162  0.602277       99      k100"
01:48:46 | INFO | "confusion matrix k100:
          anime  gaming  music  politics  sports
anime         0       0      0         1       1
gaming        0      26      0         9       3
music         0       2      1         1       2
politics      0       6      0        19       9
sports        0       2      0         2      15"


In [21]:
# with sort_labels_by_support=True, we'll see the classes with the most support (true labels) at the top 
ml_metrics.log_classification_report_and_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    class_labels=None,
    sort_labels_by_support=True,
    save_path='mlruns/logs/test_classification_report3',
    log_metrics_to_console=False,
    log_df_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_support_avg=False,
    log_support_per_class=False,
)

18:00:47 | INFO | "df metrics:
      class  precision    recall  f1_score  support data_fold
0    gaming   0.722222  0.684211  0.702703       38      k100
1  politics   0.593750  0.558824  0.575758       34      k100
2    sports   0.500000  0.789474  0.612245       19      k100
3     music   1.000000  0.166667  0.285714        6      k100
4     anime   0.000000  0.000000  0.000000        2      k100"
18:00:47 | INFO | "df metrics:
          class  precision    recall  f1_score  support data_fold
0     macro_avg   0.563194  0.439835  0.435284       99      k100
1  weighted_avg   0.637696  0.616162  0.602277       99      k100"
18:00:47 | INFO | "confusion matrix k100:
          gaming  politics  sports  music  anime
gaming        26         9       3      0      0
politics       6        19       9      0      0
sports         2         2      15      0      0
music          2         1       2      1      0
anime          0         1       1      0      0"


### Check Binary case

In [27]:
print(classification_report(y_true_binary, y_pred_binary, digits=3))

              precision    recall  f1-score   support

      gaming      1.000     0.227     0.370        22
      sports      0.819     1.000     0.901        77

    accuracy                          0.828        99
   macro avg      0.910     0.614     0.635        99
weighted avg      0.859     0.828     0.783        99



In [30]:
ml_metrics.log_classification_report_and_confusion_matrix(
    y_true=y_true_binary,
    y_pred=y_pred_binary,
    data_fold_name='kbinary',
    beta=1,
    class_labels=None,
    sort_labels_by_support=True,
    save_path='mlruns/logs/test_classification_report3',
    log_metrics_to_console=True,
    log_df_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_support_avg=False,
    log_support_per_class=False,
)

01:46:47 | INFO | "precision-sports-kbinary: 0.8191489361702128"
01:46:47 | INFO | "precision-gaming-kbinary: 1.0"
01:46:47 | INFO | "recall-sports-kbinary: 1.0"
01:46:47 | INFO | "recall-gaming-kbinary: 0.22727272727272727"
01:46:47 | INFO | "f1_score-sports-kbinary: 0.9005847953216375"
01:46:47 | INFO | "f1_score-gaming-kbinary: 0.37037037037037035"
01:46:47 | INFO | "df metrics:
    class  precision    recall  f1_score  support data_fold
0  sports   0.819149  1.000000  0.900585       77   kbinary
1  gaming   1.000000  0.227273  0.370370       22   kbinary"
01:46:48 | INFO | "precision-macro_avg-kbinary: 0.9095744680851063"
01:46:48 | INFO | "precision-weighted_avg-kbinary: 0.8593380614657211"
01:46:48 | INFO | "recall-macro_avg-kbinary: 0.6136363636363636"
01:46:48 | INFO | "recall-weighted_avg-kbinary: 0.8282828282828283"
01:46:48 | INFO | "f1_score-macro_avg-kbinary: 0.635477582846004"
01:46:48 | INFO | "f1_score-weighted_avg-kbinary: 0.7827593675546893"
01:46:48 | INFO | "df metr

## Confusion matrix

In [18]:
print(confusion_matrix(y_true_manual, y_pred_manual,))

[[ 0  0  0  1  1]
 [ 0 26  0  9  3]
 [ 0  2  1  1  2]
 [ 0  6  0 19  9]
 [ 0  2  0  2 15]]


In [33]:
ml_metrics.log_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    class_labels=None,
    save_path='mlruns/logs/test_classification_report2',
    log_df_to_console=False,
    log_artifacts_to_mlflow=False,
    sort_labels_by_support=False,
)

,anime,gaming,music,politics,sports
anime,0,0,0,1,1
gaming,0,26,0,9,3
music,0,2,1,1,2
politics,0,6,0,19,9
sports,0,2,0,2,15


In [34]:
ml_metrics.log_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    class_labels=None,
    save_path='mlruns/logs/test_classification_report2',
    log_df_to_console=False,
    log_artifacts_to_mlflow=False,
    sort_labels_by_support=True,
)

,gaming,politics,sports,music,anime
gaming,26,9,3,0,0
politics,6,19,9,0,0
sports,2,2,15,0,0
music,2,1,2,1,0
anime,0,1,1,0,0


In [20]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    class_labels=l_class_labels,
    save_path='mlruns/logs/test_classification_report2',
    log_df_to_console=False,
    log_artifacts_to_mlflow=False,
    sort_labels_by_support=True,
)

,sports,politics,gaming,anime,music
sports,15,2,2,0,0
politics,9,19,6,0,0
gaming,3,9,26,0,0
anime,1,1,0,0,0
music,2,1,2,0,1


In [119]:
ml_metrics.log_confusion_matrix(
    y_true=y_true_manual,
    y_pred=['anime'] * 49 + ['music'] * 50,
    data_fold_name='k100',
    class_labels=None,
    save_path='mlruns/logs/test_classification_report2',
    log_df_to_console=False,
    log_artifacts_to_mlflow=False,
    sort_labels_by_support=True,
)

,gaming,politics,sports,music,anime
gaming,0,0,0,23,15
politics,0,0,0,14,20
sports,0,0,0,8,11
music,0,0,0,4,2
anime,0,0,0,1,1


### Use new function to replicate classification report
Calling classification report can be messy, so try instead `precision_recall_fscore_support` which makes it easier to iterate over metrics for logging & saving.

- `accuracy` = `recall-weighted_avg`
    - so maybe don't calculate accuracy but need a way to reference that `recall-weighted_avg` is the same as accuracy



### Aggregate AND per-class metrics


In [121]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_classification_report_and_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    class_labels=l_class_labels,
    save_path='mlruns/logs/test_classification_report1',
    log_metrics_to_console=True,
    log_df_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_support_avg=True,
    log_support_per_class=True,
)

23:06:32 | INFO | "precision-sports-k100: 0.5"
23:06:32 | INFO | "precision-politics-k100: 0.59375"
23:06:32 | INFO | "precision-gaming-k100: 0.7222222222222222"
23:06:32 | INFO | "precision-anime-k100: 0.0"
23:06:32 | INFO | "precision-music-k100: 1.0"
23:06:32 | INFO | "recall-sports-k100: 0.7894736842105263"
23:06:32 | INFO | "recall-politics-k100: 0.5588235294117647"
23:06:32 | INFO | "recall-gaming-k100: 0.6842105263157895"
23:06:32 | INFO | "recall-anime-k100: 0.0"
23:06:32 | INFO | "recall-music-k100: 0.16666666666666666"
23:06:32 | INFO | "f1_score-sports-k100: 0.6122448979591837"
23:06:32 | INFO | "f1_score-politics-k100: 0.5757575757575757"
23:06:32 | INFO | "f1_score-gaming-k100: 0.7027027027027027"
23:06:32 | INFO | "f1_score-anime-k100: 0.0"
23:06:32 | INFO | "f1_score-music-k100: 0.2857142857142857"
23:06:32 | INFO | "support-sports-k100: 19"
23:06:32 | INFO | "support-politics-k100: 34"
23:06:32 | INFO | "support-gaming-k100: 38"
23:06:32 | INFO | "support-anime-k100: 2"

In [97]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_classification_report_and_confusion_matrix(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    class_labels=l_class_labels,
    save_path='mlruns/logs/test_classification_report1',
    log_metrics_to_console=False,
    log_df_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_support_avg=True,
    log_support_per_class=True,
)

22:11:28 | INFO | "df metrics:
      class  precision    recall  f1_score  support data_fold
0    sports   0.500000  0.789474  0.612245       19      k100
1  politics   0.593750  0.558824  0.575758       34      k100
2    gaming   0.722222  0.684211  0.702703       38      k100
3     anime   0.000000  0.000000  0.000000        2      k100
4     music   1.000000  0.166667  0.285714        6      k100"
22:11:28 | INFO | "df metrics:
          class  precision    recall  f1_score  support data_fold
0     macro_avg   0.563194  0.439835  0.435284       99      k100
1  weighted_avg   0.637696  0.616162  0.602277       99      k100"


### Aggregate metrics

In [92]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average='macro_and_weighted',
    class_labels=l_class_labels,
    save_path='mlruns/logs/test_classification_report',
    log_metrics_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
)

19:59:18 | INFO | "precision-macro_avg-k100: 0.5631944444444444"
19:59:18 | INFO | "precision-weighted_avg-k100: 0.6376964085297419"
19:59:18 | INFO | "recall-macro_avg-k100: 0.4398348813209495"
19:59:18 | INFO | "recall-weighted_avg-k100: 0.6161616161616161"
19:59:18 | INFO | "f1_score-macro_avg-k100: 0.4352838924267496"
19:59:18 | INFO | "f1_score-weighted_avg-k100: 0.602276758120914"


,class,precision,recall,f1_score,support,data_fold
0,macro_avg,0.563194,0.439835,0.435284,99,k100
1,weighted_avg,0.637696,0.616162,0.602277,99,k100


In [36]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average='macro_and_weighted',
    class_labels=None,
    save_path=None,
    log_metrics_to_console=False,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
    output_dict=True,
)

{'precision-macro_avg-k100': 0.5631944444444444,
 'precision-weighted_avg-k100': 0.6376964085297419,
 'recall-macro_avg-k100': 0.4398348813209495,
 'recall-weighted_avg-k100': 0.6161616161616161,
 'f1_score-macro_avg-k100': 0.4352838924267496,
 'f1_score-weighted_avg-k100': 0.602276758120914}

In [40]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average='macro_and_weighted',
    class_labels=None,
    save_path=None,
    log_metrics_to_console=False,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
    log_support=False,
    output_dict=True,
    append_fold_name_to_output_dict=False,
)

{'precision-macro_avg': 0.5631944444444444,
 'precision-weighted_avg': 0.6376964085297419,
 'recall-macro_avg': 0.4398348813209495,
 'recall-weighted_avg': 0.6161616161616161,
 'f1_score-macro_avg': 0.4352838924267496,
 'f1_score-weighted_avg': 0.602276758120914}

In [42]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average='macro_and_weighted',
    class_labels=None,
    save_path=None,
    log_metrics_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
    output_dict=True,
    append_fold_name_to_output_dict=False,
    log_support=False,
)

03:09:36 | INFO | "precision-macro_avg-k100: 0.5631944444444444"
03:09:36 | INFO | "precision-weighted_avg-k100: 0.6376964085297419"
03:09:36 | INFO | "recall-macro_avg-k100: 0.4398348813209495"
03:09:36 | INFO | "recall-weighted_avg-k100: 0.6161616161616161"
03:09:36 | INFO | "f1_score-macro_avg-k100: 0.4352838924267496"
03:09:36 | INFO | "f1_score-weighted_avg-k100: 0.602276758120914"


{'precision-macro_avg': 0.5631944444444444,
 'precision-weighted_avg': 0.6376964085297419,
 'recall-macro_avg': 0.4398348813209495,
 'recall-weighted_avg': 0.6161616161616161,
 'f1_score-macro_avg': 0.4352838924267496,
 'f1_score-weighted_avg': 0.602276758120914}

In [93]:
display(
    ml_metrics.log_precision_recall_fscore_support(
        y_true=y_true_manual,
        y_pred=y_pred_manual,
        data_fold_name='k100',
        beta=1,
        average='weighted',
        class_labels=None,
        save_path=None,
        log_metrics_to_console=True,
        log_metrics_to_mlflow=False,
        log_artifacts_to_mlflow=False,
        log_df_to_console=False,
    )
)

ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average='macro',
    class_labels=None,
    save_path=None,
    log_metrics_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
)

19:59:22 | INFO | "precision-weighted_avg-k100: 0.6376964085297419"
19:59:22 | INFO | "recall-weighted_avg-k100: 0.6161616161616161"
19:59:22 | INFO | "f1_score-weighted_avg-k100: 0.602276758120914"


,class,precision,recall,f1_score,support,data_fold
0,weighted_avg,0.637696,0.616162,0.602277,99,k100


19:59:22 | INFO | "precision-macro_avg-k100: 0.5631944444444444"
19:59:22 | INFO | "recall-macro_avg-k100: 0.4398348813209495"
19:59:22 | INFO | "f1_score-macro_avg-k100: 0.4352838924267496"


,class,precision,recall,f1_score,support,data_fold
0,macro_avg,0.563194,0.439835,0.435284,99,k100


### per-class metrics

In [90]:
print(classification_report(y_true_manual, y_pred_manual, zero_division=0, digits=3))

              precision    recall  f1-score   support

       anime      0.000     0.000     0.000         2
      gaming      0.722     0.684     0.703        38
       music      1.000     0.167     0.286         6
    politics      0.594     0.559     0.576        34
      sports      0.500     0.789     0.612        19

    accuracy                          0.616        99
   macro avg      0.563     0.440     0.435        99
weighted avg      0.638     0.616     0.602        99



In [87]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average=None,
    class_labels=l_class_labels,
    save_path=None,
    log_metrics_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
    log_support=True,
)

19:52:54 | INFO | "precision-sports-k100: 0.5"
19:52:54 | INFO | "precision-politics-k100: 0.59375"
19:52:54 | INFO | "precision-gaming-k100: 0.7222222222222222"
19:52:54 | INFO | "precision-anime-k100: 0.0"
19:52:54 | INFO | "precision-music-k100: 1.0"
19:52:54 | INFO | "recall-sports-k100: 0.7894736842105263"
19:52:54 | INFO | "recall-politics-k100: 0.5588235294117647"
19:52:54 | INFO | "recall-gaming-k100: 0.6842105263157895"
19:52:54 | INFO | "recall-anime-k100: 0.0"
19:52:54 | INFO | "recall-music-k100: 0.16666666666666666"
19:52:54 | INFO | "f1_score-sports-k100: 0.6122448979591837"
19:52:54 | INFO | "f1_score-politics-k100: 0.5757575757575757"
19:52:54 | INFO | "f1_score-gaming-k100: 0.7027027027027027"
19:52:54 | INFO | "f1_score-anime-k100: 0.0"
19:52:54 | INFO | "f1_score-music-k100: 0.2857142857142857"
19:52:54 | INFO | "support-sports-k100: 19"
19:52:54 | INFO | "support-politics-k100: 34"
19:52:54 | INFO | "support-gaming-k100: 38"
19:52:54 | INFO | "support-anime-k100: 2"

,class,precision,recall,f1_score,support,data_fold
0,sports,0.500000,0.789474,0.612245,19,k100
1,politics,0.593750,0.558824,0.575758,34,k100
2,gaming,0.722222,0.684211,0.702703,38,k100
3,anime,0.000000,0.000000,0.000000,2,k100
4,music,1.000000,0.166667,0.285714,6,k100


In [88]:
l_class_labels = ['sports', 'politics', 'gaming', 'anime', 'music']
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average=None,
    class_labels=l_class_labels,
    save_path='mlruns/logs/test_classification_report',
    log_metrics_to_console=True,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
    log_support=True,
)

19:55:50 | INFO | "precision-sports-k100: 0.5"
19:55:50 | INFO | "precision-politics-k100: 0.59375"
19:55:50 | INFO | "precision-gaming-k100: 0.7222222222222222"
19:55:50 | INFO | "precision-anime-k100: 0.0"
19:55:50 | INFO | "precision-music-k100: 1.0"
19:55:50 | INFO | "recall-sports-k100: 0.7894736842105263"
19:55:50 | INFO | "recall-politics-k100: 0.5588235294117647"
19:55:50 | INFO | "recall-gaming-k100: 0.6842105263157895"
19:55:50 | INFO | "recall-anime-k100: 0.0"
19:55:50 | INFO | "recall-music-k100: 0.16666666666666666"
19:55:50 | INFO | "f1_score-sports-k100: 0.6122448979591837"
19:55:50 | INFO | "f1_score-politics-k100: 0.5757575757575757"
19:55:50 | INFO | "f1_score-gaming-k100: 0.7027027027027027"
19:55:50 | INFO | "f1_score-anime-k100: 0.0"
19:55:50 | INFO | "f1_score-music-k100: 0.2857142857142857"
19:55:50 | INFO | "support-sports-k100: 19"
19:55:50 | INFO | "support-politics-k100: 34"
19:55:50 | INFO | "support-gaming-k100: 38"
19:55:50 | INFO | "support-anime-k100: 2"

,class,precision,recall,f1_score,support,data_fold
0,sports,0.500000,0.789474,0.612245,19,k100
1,politics,0.593750,0.558824,0.575758,34,k100
2,gaming,0.722222,0.684211,0.702703,38,k100
3,anime,0.000000,0.000000,0.000000,2,k100
4,music,1.000000,0.166667,0.285714,6,k100


In [84]:
ml_metrics.log_precision_recall_fscore_support(
    y_true=y_true_manual,
    y_pred=y_pred_manual,
    data_fold_name='k100',
    beta=1,
    average=None,
    class_labels=None,
    save_path=None,
    log_metrics_to_console=False,
    log_metrics_to_mlflow=False,
    log_artifacts_to_mlflow=False,
    log_df_to_console=False,
)

,class,precision,recall,f1_score,support,data_fold
0,class_0,0.000000,0.000000,0.000000,2,k100
1,class_1,0.722222,0.684211,0.702703,38,k100
2,class_2,1.000000,0.166667,0.285714,6,k100
3,class_3,0.593750,0.558824,0.575758,34,k100
4,class_4,0.500000,0.789474,0.612245,19,k100


In [15]:
lbls_ = ['gaming', 'politics', 'sports']
precision_recall_fscore_support(y_true, y_pred, beta=1, labels=lbls_, average=None)

(array([0.88235294, 0.51282051, 0.61538462]),
 array([0.6122449 , 0.76923077, 0.66666667]),
 array([0.72289157, 0.61538462, 0.64      ]),
 array([49, 26, 24]))

If we supply the labels, the order of the arrays will match the labels and we can use this to create a dict, which we can then convert into a dataframe.

In [16]:
d_class_metrics = dict()
col_class_labels = 'class'
beta = 1
d_class_metrics[col_class_labels] = ['gaming', 'sports', 'politics',]
(
    d_class_metrics['precision'],
    d_class_metrics['recall'],
    d_class_metrics[f'f{beta}_score'],
    d_class_metrics['support']
) = precision_recall_fscore_support(y_true, y_pred, beta=beta, labels=d_class_metrics[col_class_labels], average=None)
display(pd.DataFrame(d_class_metrics))

for metric_ in [k for k in d_class_metrics.keys() if k != col_class_labels]:
    for class_, val_ in zip(d_class_metrics[col_class_labels], d_class_metrics[metric_]):
        print(f"{metric_}-{class_}", val_)

,class,precision,recall,f1_score,support
0,gaming,0.882353,0.612245,0.722892,49
1,sports,0.615385,0.666667,0.640000,24
2,politics,0.512821,0.769231,0.615385,26


precision-gaming 0.8823529411764706
precision-sports 0.6153846153846154
precision-politics 0.5128205128205128
recall-gaming 0.6122448979591837
recall-sports 0.6666666666666666
recall-politics 0.7692307692307693
f1_score-gaming 0.7228915662650602
f1_score-sports 0.64
f1_score-politics 0.6153846153846154
support-gaming 49
support-sports 24
support-politics 26


In [54]:
log_clf_report_and_conf_matrix(
        y_true=y_true,
        y_pred=y_pred,
        data_fold_name='k2',
        class_labels=['gaming', 'politics', 'sports'],
        save_path=None,
        log_metrics_to_mlflow=False,
        log_artifacts_to_mlflow=False,
        log_to_console=True,
        remove_files_from_local_path=False,
        print_clf_df=True,
        return_confusion_mx=False,
)

05:41:59 | INFO | "Start processing K2 metrics for logging"
  0%|          | 0/15 [00:00<?, ?it/s]05:41:59 | INFO | " K2 gaming-f1_score: 0.816327"
05:41:59 | INFO | " K2 gaming-support: 50"
05:41:59 | INFO | " K2 politics-f1_score: 0.677419"
05:41:59 | INFO | " K2 politics-support: 36"
05:41:59 | INFO | " K2 sports-f1_score: 0.526316"
05:41:59 | INFO | " K2 sports-support: 13"
05:41:59 | INFO | " K2 accuracy: 0.717172"
05:41:59 | INFO | " K2 precision_macro_avg: 0.680342"
05:41:59 | INFO | " K2 recall_macro_avg: 0.717521"
05:41:59 | INFO | " K2 f1_score_macro_avg: 0.673354"
05:41:59 | INFO | " K2 support_macro_avg: 99"
05:41:59 | INFO | " K2 precision_weighted_avg: 0.767107"
05:41:59 | INFO | " K2 recall_weighted_avg: 0.717172"
05:41:59 | INFO | " K2 f1_score_weighted_avg: 0.727733"
05:41:59 | INFO | " K2 support_weighted_avg: 99"
100%|██████████| 15/15 [00:00<00:00, 1152.53it/s]

             precision recall  f1-score  support
class                                           
gaming           0.833  0.800     0.816       50
politics         0.808  0.583     0.677       36
sports           0.400  0.769     0.526       13
macro avg        0.680  0.718     0.673       99
weighted avg     0.767  0.717     0.728       99
accuracy                          0.717       99


In [25]:
d_clf_report = dict()
d_clf_report['class'] = ['sports', 'gaming', 'politics', ]
(
    d_clf_report['precision'],
    d_clf_report['recall'],
    d_clf_report['f1_score'],
    d_clf_report['support']
) = precision_recall_fscore_support(y_true, y_pred, beta=1, labels=d_clf_report['class'], average=None)
pd.DataFrame(d_clf_report)

,class,precision,recall,f1_score,support
0,sports,0.400000,0.769231,0.526316,13
1,gaming,0.833333,0.800000,0.816327,50
2,politics,0.807692,0.583333,0.677419,36


In [11]:
precision_recall_fscore_support(y_true, y_pred, beta=1, labels=['politics', 'sports', 'gaming', ])

(array([0.66666667, 0.58823529, 0.8       ]),
 array([0.8       , 0.76923077, 0.58333333]),
 array([0.72727273, 0.66666667, 0.6746988 ]),
 array([25, 26, 48]))

# Sort metrics by max value
Use it when plotting metrics columns so that legend better matches the max vals

In [45]:
from io import StringIO

In [64]:
sample_csv = StringIO(""",predicted_col,truth_col,k,precision-macro_avg,precision-weighted_avg,recall-macro_avg,recall-weighted_avg,f1_score-macro_avg,f1_score-weighted_avg,adjusted_mutual_info_score,adjusted_rand_score,homogeneity_score
0,k_0010_label,primary_topic,10,0.0516806188445924,0.2817722277267966,0.1082339054144935,0.43365287588294654,0.06505597710874912,0.328738689674732,0.4012787579282346,0.38748295082592166,0.31599968245603843
2,k_0020_label,primary_topic,20,0.1460659256441579,0.44454879342261516,0.206781144238916,0.5370837537840565,0.1588967053468496,0.4715698043958237,0.5058768115133843,0.5593907251630248,0.44908113539076533
3,k_0027_label,primary_topic,27,0.1930505119805911,0.4702203007968436,0.2649275871532684,0.5804742684157417,0.21278190187058516,0.5079092557205998,0.5308927985385036,0.5606958021292359,0.489011874523237
4,k_0030_label,primary_topic,30,0.19288497682596695,0.46812373191245854,0.2656414158016345,0.5817356205852674,0.2134600789989808,0.5083410613593178,0.5328450572548121,0.5613303619963432,0.49123226722963376
5,k_0040_label,primary_topic,40,0.2548669681579302,0.5387695395078037,0.3111553029752653,0.6092330978809284,0.2646164081386312,0.557560309526615,0.5497138565347913,0.6042089123957942,0.5224587708904974
6,k_0050_label,primary_topic,50,0.3094331567452062,0.5803769604502006,0.35993161622177267,0.6228557013118062,0.31495135166828947,0.5869067469372687,0.5611010494741004,0.6342021442446079,0.5523380994990401
7,k_0060_label,primary_topic,60,0.33337334723020995,0.5933788594942602,0.3683829966446991,0.6326942482341069,0.33455289297721846,0.600943856848398,0.5652372333659587,0.6388571694269723,0.5607311332882817
8,k_0062_label,primary_topic,62,0.33826793960186297,0.5931643685451574,0.36797311208847316,0.6349646821392533,0.3382625285110353,0.6021507224435417,0.5653424973032566,0.6380064424191283,0.5595750577972479
9,k_0070_label,primary_topic,70,0.3459810294741316,0.5943854778141853,0.3703449162200727,0.6465691220988901,0.3443241479312507,0.6084211752241301,0.5711723370680776,0.63863545175157,0.5598712035039742
10,k_0080_label,primary_topic,80,0.40386217464843494,0.6340087117644443,0.4044604317655758,0.6632189707366297,0.38849671606946057,0.6339910141919486,0.5803213281222485,0.6564260726341998,0.577419648656207
11,k_0090_label,primary_topic,90,0.42615493007717886,0.645477821514172,0.41289893432947766,0.6680121089808274,0.4032112808669569,0.6414317437885179,0.5834126273314906,0.6567152940294144,0.5810722092322408
12,k_0092_label,primary_topic,92,0.42647418634583095,0.6461610069545165,0.4135980394748915,0.6692734611503531,0.4033951027910398,0.6417695605459236,0.5842411866668369,0.6575991305573501,0.5816572854478811
13,k_0100_label,primary_topic,100,0.4420951610192523,0.6494306454994127,0.4305221546563613,0.6730575176589304,0.41915815649126664,0.6450096808166197,0.5871093122936466,0.6584889352847391,0.5851643146859925
14,k_0110_label,primary_topic,110,0.4508711978387056,0.6558410122669128,0.437395293226388,0.6788597376387487,0.42854532993952305,0.652545739233146,0.5872885640633062,0.6608128392840327,0.5875261142862265
15,k_0125_label,primary_topic,125,0.4554283553772779,0.656917940397822,0.4461494714264962,0.6846619576185671,0.4366899546263309,0.657060026244786,0.588918317876023,0.6603856016100269,0.589337045161037
16,k_0150_label,primary_topic,150,0.5279286342713212,0.6831180508740841,0.4939589721816706,0.6955095862764884,0.49173664382267557,0.6745049152414666,0.5971618576997808,0.6794122224584846,0.6056461342799535
17,k_0175_label,primary_topic,175,0.5322183381798214,0.6871948020959623,0.49917218615312636,0.7033299697275479,0.5005079208722945,0.6834352472885595,0.6028586624563061,0.6879137462666625,0.6112194164591204
18,k_0200_label,primary_topic,200,0.5449805333774532,0.6917494318010378,0.5096795529290102,0.7078708375378405,0.512392345997014,0.688487198117112,0.6065260940415947,0.6916278732624854,0.616149354070536
19,k_0300_label,primary_topic,300,0.5990142153265386,0.7160817527240095,0.5630581994255957,0.731079717457114,0.5683150465260515,0.7148991988885902,0.6210653995968108,0.7015299257873947,0.632765110792565
20,k_0400_label,primary_topic,400,0.614481548865005,0.7282804021417887,0.5788966350352317,0.742936427850656,0.5883849411382307,0.7306375044145708,0.632853624859808,0.7186667658798723,0.6460525408309405
""")

In [65]:
df_metrics = pd.read_table(sample_csv, sep=',', skip_blank_lines=True)
df_metrics.shape

(20, 13)

In [66]:
df_metrics.head()

,Unnamed: 0,predicted_col,truth_col,k,precision-macro_avg,precision-weighted_avg,recall-macro_avg,recall-weighted_avg,f1_score-macro_avg,f1_score-weighted_avg,adjusted_mutual_info_score,adjusted_rand_score,homogeneity_score
0,0,k_0010_label,primary_topic,10,0.051681,0.281772,0.108234,0.433653,0.065056,0.328739,0.401279,0.387483,0.316000
1,2,k_0020_label,primary_topic,20,0.146066,0.444549,0.206781,0.537084,0.158897,0.471570,0.505877,0.559391,0.449081
2,3,k_0027_label,primary_topic,27,0.193051,0.470220,0.264928,0.580474,0.212782,0.507909,0.530893,0.560696,0.489012
3,4,k_0030_label,primary_topic,30,0.192885,0.468124,0.265641,0.581736,0.213460,0.508341,0.532845,0.561330,0.491232
4,5,k_0040_label,primary_topic,40,0.254867,0.538770,0.311155,0.609233,0.264616,0.557560,0.549714,0.604209,0.522459


In [67]:
metric_cols = [
    c for c in df_metrics.columns if
    c.endswith('_avg')
]
metric_cols

['precision-macro_avg',
 'precision-weighted_avg',
 'recall-macro_avg',
 'recall-weighted_avg',
 'f1_score-macro_avg',
 'f1_score-weighted_avg']

In [68]:
df_metrics[metric_cols].max().sort_values(ascending=False)

recall-weighted_avg       0.742936
f1_score-weighted_avg     0.730638
precision-weighted_avg    0.728280
precision-macro_avg       0.614482
f1_score-macro_avg        0.588385
recall-macro_avg          0.578897
dtype: float64

In [69]:
df_metrics[metric_cols].max().sort_values(ascending=False).index.to_list()

['recall-weighted_avg',
 'f1_score-weighted_avg',
 'precision-weighted_avg',
 'precision-macro_avg',
 'f1_score-macro_avg',
 'recall-macro_avg']